# Capstone Project - The Battle of Neighborhoods (Week 1-2)

## Introduction

### Background

London is considered to be one of the world's most important global cities and has been called the world's most powerful, most desirable, most influential, most visited, most expensive, innovative, sustainable, most investment-friendly, and most-popular-for-work city.

London attract lots of new inhabitants, new investors each year because above reasons.

### Problem

London is a huge city, not everyone known London well before they go there. It is risky to move to a place you barely known.

This report addressed this problem and provide a investment guide to people who want to open a shop or restaurant in London. 

It supposed to help those people to have a general business view of London. So, they can better decide where they are going.

### Assumptions

Usually businesses are grouped together, people like to go to clustered shops or restaurant to have a wider range of choices hence more efficient shopping/eating experience. If you want to open a shop/restaurant, I assume it is safer to go to those places which already have shops/restaurants to minimize the risk.

Besides that, richer neighborhoods usually have higher spending power, you would like to setup your business there to maximize the revenue. In this case, I assume higher house sales price will indicate higher spending power.

So, this report are based on two assumptions:
1. More similar business at the same place indicates lesser risk.
2. Higher average house sales price indicates higher revenue.


## Data

To achieve the goal of minimal risk and maximal revenue. I need different sources of data.

- List of boroughs can be obtained on Wikipedia. https://en.wikipedia.org/wiki/List_of_London_boroughs
    - Borough names and geo coordinates can be found here, geo coordinates will be used in foursquare API to find recommended venues.
- Areas list on Wikipedia https://en.wikipedia.org/wiki/List_of_areas_of_London
    - Postcode to boroughs mapping can be extracted here, Postcode will be used in house sales data to find house price of that postcode then that borough.
- By using foursquare API, list of recommended venues can be fetched base on above coordinates. https://api.foursquare.com/v2/venues/explore
    - List of top venues of each borough can be found here to decide the functionality of a borough
- House sale price data on Land registry of UK, I will use most recent data from year 2019. http://prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv
    - Postcode to house sale price mapping can be extracted here.


In [ ]:
#!wget http://prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv

In [14]:
# The code was removed by Watson Studio for sharing.

In [15]:
df_data_0 = pd.read_csv(body, names=['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', 'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status'])
df_data_0.head()
df_london = df_data_0[df_data_0['Town_City'] == 'LONDON']
df_london.shape


(57316, 16)

In [16]:
df_london.head()

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
285,{8CAC1318-AFF3-0253-E053-6B04A8C08E51},1100000,2019-06-07 00:00,SW19 8BN,T,N,F,30,NaN,ASHEN GROVE,NaN,LONDON,MERTON,GREATER LONDON,A,A
287,{8CAC1318-AFF5-0253-E053-6B04A8C08E51},905000,2019-06-20 00:00,SW19 8BH,T,N,F,35,NaN,DURNSFORD AVENUE,NaN,LONDON,MERTON,GREATER LONDON,A,A
294,{8CAC1318-AFFC-0253-E053-6B04A8C08E51},1440000,2019-06-07 00:00,SW13 8DL,T,N,F,29,NaN,MERTHYR TERRACE,NaN,LONDON,RICHMOND UPON THAMES,GREATER LONDON,A,A
382,{8355F009-B3C5-55C5-E053-6B04A8C0D090},1800000,2019-02-04 00:00,SW1V 4NU,T,N,F,89,NaN,WINCHESTER STREET,NaN,LONDON,CITY OF WESTMINSTER,GREATER LONDON,B,A
489,{8F1B26BD-4595-53DB-E053-6C04A8C03649},340000,2019-07-15 00:00,N1 9BT,F,N,L,75,FLAT 4,CALEDONIAN ROAD,NaN,LONDON,ISLINGTON,GREATER LONDON,A,A
